In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
# Preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
#Metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix


In [34]:
df = pd.read_csv('/content/marketing_conversion_synth.csv')

In [35]:
df.head()

,session_id,user_id,age,device,traffic_source,gender,country,time_on_site,pageviews,previous_visits,cart_value,days_since_last_visit,has_discount,conversion
0,session_00000,180325,60,desktop,direct,male,ES,35,3,19,154.25,21,0,0
1,session_00001,796559,20,mobile,email,male,Other,437,13,10,281.61,31,0,0
2,session_00002,689113,37,tablet,social,female,FR,140,9,12,314.66,78,0,0
3,session_00003,494990,39,desktop,ads,male,FR,431,1,7,193.08,9,0,0
4,session_00004,489713,56,mobile,seo,male,UK,345,11,18,170.17,9,0,0


# We need to convert all categorical values into numerical values by encoding them

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   session_id             2000 non-null   object 
 1   user_id                2000 non-null   int64  
 2   age                    2000 non-null   int64  
 3   device                 2000 non-null   object 
 4   traffic_source         2000 non-null   object 
 5   gender                 2000 non-null   object 
 6   country                2000 non-null   object 
 7   time_on_site           2000 non-null   int64  
 8   pageviews              2000 non-null   int64  
 9   previous_visits        2000 non-null   int64  
 10  cart_value             2000 non-null   float64
 11  days_since_last_visit  2000 non-null   int64  
 12  has_discount           2000 non-null   int64  
 13  conversion             2000 non-null   int64  
dtypes: float64(1), int64(8), object(5)
memory usage: 218.9+ 

In [37]:
df["conversion"].value_counts(normalize = True)

,proportion
conversion,
0,0.8535
1,0.1465


In [38]:
df.columns

Index(['session_id', 'user_id', 'age', 'device', 'traffic_source', 'gender',
       'country', 'time_on_site', 'pageviews', 'previous_visits', 'cart_value',
       'days_since_last_visit', 'has_discount', 'conversion'],
      dtype='object')

In [39]:
# Target

target = "conversion"
# We can also use this instead of that y = df["conversion"]


#numeric features
num_col = [
    "age",
    "time_on_site",
    "pageviews",
    "previous_visits",
    "cart_value",
    "days_since_last_visit",
    "has_discount"
]


#Categorical features
cat_col = [
    'device',
    'traffic_source',
    'gender',
    'country'
]


#Drop useless ID cols

df = df.drop (["session_id", "user_id"], axis = 1)
# Axis = 1 is used for dropping columns
# Axis = 0 is used for dropping rows

In [40]:
df.sample(10)

,age,device,traffic_source,gender,country,time_on_site,pageviews,previous_visits,cart_value,days_since_last_visit,has_discount,conversion
302,45,mobile,social,female,FR,474,6,9,386.53,24,0,0
315,18,mobile,seo,female,DE,478,14,15,118.59,32,1,1
1951,28,tablet,email,female,Other,178,4,1,282.21,57,1,0
1730,52,tablet,ads,female,FR,433,11,13,298.03,14,0,1
1559,40,mobile,ads,female,ES,299,15,16,121.06,35,0,0
709,59,desktop,ads,male,FR,291,7,4,63.59,8,0,1
1896,43,mobile,ads,female,ES,302,9,4,24.47,44,1,0
1337,20,desktop,seo,male,DE,120,7,8,331.70,22,0,1
584,34,mobile,seo,male,UK,335,13,11,209.00,48,1,0
1875,46,mobile,email,male,FR,376,15,6,279.35,52,0,1


In [41]:
feature_cols = num_col + cat_col

X = df[feature_cols]
Y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2,
                                                    stratify = Y, random_state = 42)

print ("Here is the dimensions after training")
print ("Shape of X_train is - ", X_train.shape)
print ("Shape of X_test is - ", X_test.shape)
print ("Shape of Y_train is - ", y_train.shape)
print ("Shape of Y_test is - ", y_test.shape)

Here is the dimensions after training
Shape of X_train is -  (1600, 11)
Shape of X_test is -  (400, 11)
Shape of Y_train is -  (1600,)
Shape of Y_test is -  (400,)


# Building a pipeline for the ML model

In [42]:
# 1. Define transformations for each type

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown = "ignore")

# 2. Column transformer - Tells sklearn what to apply to which tool

preprocessor = ColumnTransformer(
    transformers = [
        ("num", numeric_transformer, num_col),
        ("cat", categorical_transformer, cat_col)
    ]
)

# 3. Full Pipeline: preprocessing + model

pipeline = Pipeline(
    steps = [
        ("preprocessor", preprocessor),
        ("classifier", LogisticRegression(max_iter = 1000, class_weight='balanced'))
    ]
)



In [43]:
# Fit the pipeline

pipeline.fit (X_train, y_train)

print ("Model has fitted succesfully")


Model has fitted succesfully


In [44]:
# Predict Probabilities

proba = pipeline.predict_proba(X_test)[:, 1]
# Probability of conversion (Class 1)

# Predicts labels (0 or 1) on the test set
y_pred = pipeline.predict(X_test)

#Comparing both

print("predicted labels (0/1): ")
print(y_pred[:10]) # show first 10 elements for better clarity

print(" \n predicted probabilities: ")
print(proba[:10]) # Show first 10 elements for better clarity





predicted labels (0/1): 
[0 0 0 0 0 0 0 0 0 0]
 
 predicted probabilities: 
[0.00055448 0.01941345 0.00263677 0.06052389 0.00403274 0.12571228
 0.02733663 0.02430254 0.0305545  0.00378948]


In [45]:
#Evaluate Metrics

print ("Accuracy: ", round(accuracy_score(y_test, y_pred),4))
print ("Precision: ", round(precision_score(y_test, y_pred),4))
print ("Recall: ", round(recall_score(y_test, y_pred),4))
print ("F1 Score: ", round(f1_score(y_test, y_pred),4))

print ("\n")
print ("\n")

print ("Accuracy: ", round(accuracy_score(y_test, y_pred),4))
print ("Precision: ", round(precision_score(y_test, y_pred),4))
print ("Recall: ", round(recall_score(y_test, y_pred),4))
print ("F1: ", round(f1_score(y_test, y_pred),4))


print ("Accuracy: ", round(accuracy_score(y_test, y_pred_thresh),4))
print ("Precision: ", round(precision_score(y_test, y_pred_thresh),4))
print ("Recall: ", round(recall_score(y_test, y_pred_thresh),4))
print ("F1 score: ", round(f1_score(y_test, y_pred_thresh),4))

Accuracy:  0.8775
Precision:  0.6136
Recall:  0.4576
F1 Score:  0.5243




Accuracy:  0.8775
Precision:  0.6136
Recall:  0.4576
F1:  0.5243
Accuracy:  0.88
Precision:  0.8667
Recall:  0.2203
F1 score:  0.3514


In [46]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[324  17]
 [ 32  27]]


In [47]:
# Test thresholds
thresholds = [0.30, 0.50, 0.70]
for t in thresholds:
  y_pred_thresh = (proba >= t).astype(int)
print(f"\nThreshold = {t}")
print("Accuracy:", round(accuracy_score(y_test, y_pred_thresh), 4))
print("Precision:", round(precision_score(y_test, y_pred_thresh), 4))
print("Recall :", round(recall_score(y_test, y_pred_thresh), 4))


print("F1-score :", round(f1_score(y_test, y_pred_thresh), 4))


Threshold = 0.7
Accuracy: 0.88
Precision: 0.8667
Recall : 0.2203
F1-score : 0.3514


In [47]:
# 3. Full Pipeline: preprocessing + model
# Added 'class_weight="balanced"' to prioritize the minority class (converters)
pipeline = Pipeline(
    steps = [
        ("preprocessor", preprocessor),
        ("classifier", LogisticRegression(max_iter = 1000, class_weight='balanced'))
    ]
)

In [50]:
print ("Accuracy: ", round(accuracy_score(y_test, y_pred),4))
print ("Precision: ", round(precision_score(y_test, y_pred),4))
print ("Recall: ", round(recall_score(y_test, y_pred),4))
print ("F1 Score: ", round(f1_score(y_test, y_pred),4))

Accuracy:  0.8775
Precision:  0.6136
Recall:  0.4576
F1 Score:  0.5243


# Executive Summary & Recommendations

### 1. Model Performance (Propensity to Buy)
* **Accuracy:** ~88% (The model correctly predicts the outcome most of the time).
* **Recall:** This is our most critical metric. At our standard threshold, we identify **46%** of all converting customers.
* **Precision:** Of the people we predicted would buy, **61%** actually did.

### 2. The Trade-off: Precision vs. Recall
We tested different "Aggressiveness Thresholds" (0.3, 0.5, 0.7) to see how they impact campaign performance:

* **Aggressive Strategy (Threshold 0.3):**
    * **Recall increases to ~63%.** We capture more potential buyers.
    * **Trade-off:** Precision drops. We will target more people who *don't* buy (False Positives).
    * *Use Case:* High-margin products where missing a sale is expensive, but sending an email is cheap.

* **Conservative Strategy (Threshold 0.7):**
    * **Precision jumps to ~87%.** Almost everyone we target buys.
    * **Trade-off:** Recall drops to ~22%. We miss many buyers.
    * *Use Case:* Expensive direct mail campaigns or limited-stock items where we only want to target "sure things."

### 3. Business Recommendation
* **For Email Marketing:** Use a **0.3 Threshold**. Since emails are cheap, we should cast a wider net to capture 63% of converters, even if it means emailing some non-buyers.
* **For Paid Retargeting:** Use a **0.5 or 0.7 Threshold**. Ad spend is expensive; restrict budget to users with a high probability (>50%) of converting.